In [188]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
import pickle

In [189]:
df = pd.read_csv('Churn_Modelling.csv', sep=',')
df.head(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [190]:
df = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [191]:
one_hot_encoder = OneHotEncoder(handle_unknown='ignore')
label_gender_encoder = LabelEncoder()
standard_scaler = StandardScaler()
df['Gender'] = label_gender_encoder.fit_transform(df['Gender'])

In [192]:
geo_encoded = one_hot_encoder.fit_transform(df[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=one_hot_encoder.get_feature_names_out(['Geography']))
df = pd.concat([df, geo_encoded_df], axis=1)

In [193]:
X = df.drop(['EstimatedSalary', 'Geography'], axis=1)

y = df['EstimatedSalary']
X = standard_scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [194]:
with open('one_hot_encoder.pkl', 'wb') as file:
    pickle.dump(one_hot_encoder, file)
with open('label_gender_encoder.pkl', 'wb') as file:
    pickle.dump(label_gender_encoder, file)
with open('standard_scaler.pkl', 'wb') as file:
    pickle.dump(standard_scaler, file)

In [195]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import datetime
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

In [196]:
model = Sequential(
    [
        Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
        Dense(1)
    ]
)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])


/Users/tafehi/personal/github/ann-regression-salary/.venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [197]:
model.summary()

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_34 (Dense)                │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_35 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_36 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_37 (Dense)                │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,457 (13.50 KB)

 Trainable params: 3,457 (13.50 KB)

 Non-trainable params: 0 (0.00 B)

In [198]:
# Set up the TensorBoard callback
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [199]:
## Early stopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

In [184]:
## Train the model
history = model.fit(X_train, y_train, 
                    validation_data=(X_test, y_test), 
                    epochs=100, 
                    callbacks=[tensorflow_callback, early_stopping_callback]) 

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 13366487040.0000 - mean_squared_error: 13366487040.0000 - val_loss: 12877759488.0000 - val_mean_squared_error: 12877759488.0000
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 736us/step - loss: 13160518656.0000 - mean_squared_error: 13160518656.0000 - val_loss: 10182019072.0000 - val_mean_squared_error: 10182019072.0000
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 690us/step - loss: 8899983360.0000 - mean_squared_error: 8899983360.0000 - val_loss: 4640165376.0000 - val_mean_squared_error: 4640165376.0000
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 692us/step - loss: 4034733056.0000 - mean_squared_error: 4034733056.0000 - val_loss: 3480721408.0000 - val_mean_squared_error: 3480721408.0000
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 647us/step - loss: 3427552000.0000 - mean_squared_error: 3427552000.0000 - val_loss: 3425577984.0000 - val_mean_squared_error: 3425577984.0000
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 697us/step - lo

In [185]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [186]:
# The tensorboard extension is already loaded. To reload it, use:
%reload_ext tensorboard

In [187]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 36397), started 0:01:12 ago. (Use '!kill 36397' to kill it.)

In [200]:
model.save('regression_model.h5')